In [1]:
#!conda install biopython --yes

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/alex/anaconda3/envs/py35:
#
biopython                 1.68                np112py35_0  


In [2]:
from Bio.SeqUtils import IUPACData
list(map(str.upper,IUPACData.protein_letters_3to1.keys()))

['ASP',
 'ILE',
 'GLN',
 'HIS',
 'GLY',
 'ASN',
 'LEU',
 'CYS',
 'GLU',
 'SER',
 'TRP',
 'ARG',
 'VAL',
 'MET',
 'THR',
 'TYR',
 'PHE',
 'PRO',
 'ALA',
 'LYS']

In [7]:
import rosetta
import pyrosetta

INPUT_POSE = 'input_pose.pdb'
PARAMS = 'pnpg.params'
CST_FILE = 'pnpg.cst'

pyrosetta.init('-beta -extra_res_fa pnpg.params -run:preserve_header')
p = pyrosetta.pose_from_file(INPUT_POSE)
sfxn = pyrosetta.create_score_function('beta_cst')
soft_rep = pyrosetta.create_score_function('soft_rep')

# mm = pyrosetta.MoveMap()
# mm.show(p.total_residue())

addcsts = rosetta.protocols.enzdes.AddOrRemoveMatchCsts()
addcsts.cstfile(CST_FILE) 
addcsts.set_cst_action(rosetta.protocols.enzdes.CstAction.ADD_NEW)
addcsts.apply(p)

Found rosetta database at: /Users/alex/anaconda3/envs/py35/lib/python3.5/site-packages/pyrosetta-4.0-py3.5.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:06c81a707244fa2d0558e2ddaab9e620acc86f1e 2017-04-25 21:54:22 -0700] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [8]:
mutate = rosetta.protocols.simple_moves.MutateResidue(70, 'PHE')
mutate.apply(p)

In [9]:
p.sequence()[69]

'F'

In [10]:
# output a PDB file 
p.dump_scored_pdb('input_pose_0001.pdb', sfxn)

In [11]:
! ls 

SJB_enzdes_notebook.ipynb        pnpg.cst
data_processing.py               pnpg.params
input_pose.pdb                   protocol.py
input_pose_0001.pdb              protocol.py.bak
minimal_pyrosetta_notebook.ipynb sub.sh


In [ ]:
# reference 




# predock = rosetta.protocols.enzdes.PredesignPerturbMover()
# predock.trans_magnitude(0.1)
# predock.rot_magnitude(2)
# predock.set_ligand(p.n_total()) 

# ## These two objects/movers are the business end of enzdes in rosetta. They setup the object, some options,
# ## with the only difference here being the second one turns backbone (bb) minimziations on
# enzdes = rosetta.protocols.enzdes.EnzRepackMinimize()
# enzdes.set_scorefxn_minimize(sfxn)
# enzdes.set_scorefxn_repack(soft_rep)
# enzdes.set_min_lig(True)
# enzdes.set_min_rb(True)
# enzdes.set_min_sc(True)
# enzdes.set_design(False)

# enzdes_wbb = rosetta.protocols.enzdes.EnzRepackMinimize()
# enzdes_wbb.set_scorefxn_minimize(sfxn)
# enzdes_wbb.set_min_lig(True)
# enzdes_wbb.set_min_rb(True)
# enzdes_wbb.set_min_sc(True)
# enzdes_wbb.set_design(False)
# enzdes_wbb.set_scorefxn_repack(soft_rep)
# enzdes_wbb.set_min_bb(True)

# ## Notes for the future

# # min in stages?  # further investigation required here
# # backrub? # further investigation required here

# # Now score the protein (this includes the constraints in the score)
# sfxn(p) # you need this for nbr graph to populate
# # Setup task opertations and add them to the enzdes movers

# tf = rosetta.core.pack.task.TaskFactory()

# # Sets up the packing/design shells from the global options set above
# dp = rosetta.protocols.enzdes.DetectProteinLigandInterface()
# dp.init_from_options()

# ## Applies the enigmatic Task Operation LimitAromaticChi2 operation to the packer task.
# ## This is one of those legacy operations that you have to ask the Rosetta elders to explain
# ## Rosetta source credits Nobuyasu for this TO
# limchi2 = rosetta.protocols.toolbox.task_operations.LimitAromaChi2Operation()

# # This restricts the residues define in the constraint file to only be allowed to pack, not designable
# canttouchcatres = rosetta.protocols.enzdes.SetCatalyticResPackBehavior()
# canttouchcatres.set_fix_catalytic_aa(False) ## seems to freeze them, no repacking if set to True

# # Push back all of our TO's onto the task factory (see Design Patterns on Factorys)
# tf.push_back(dp)
# tf.push_back(canttouchcatres)
# tf.push_back(limchi2)

# # Create a packer task, specifically for the cstopt mover to work
# pt = tf.create_task_and_apply_taskoperations(p)

# enzdes.task_factory( tf ) # sets the enzdes movers to use our task factory set with our task operations
# enzdes_wbb.task_factory( tf )
# # Just leaving this here as an example, but this allows for manual inspection of your packer task
# # after all of the task operations have been applied. You could also apply one at a time to verify how
# # they are working and that they are indeed working.

# # for i in range(1,p.total_residue()+1):
# #     print(i,pt.being_designed(i),pt.being_packed(i))
# print(pt)
# # Warning, what follows is a bunch of **** all because there is not a getter/setter
# # for cst_opt in the enzdesmovers

# # Mover sub-classing -----------------------------------
# # Thank you Sergey!
# #
# class My_New_Mover(rosetta.protocols.moves.Mover):
#     def __init__(self,sfxn,pt):
#         print( 'My_New_Mover.__init__...' )
#         rosetta.protocols.moves.Mover.__init__(self)
#         self.sfxn = sfxn
#         self.pt = pt

#     def get_name(self): return 'My_New_Mover'

#     def apply(self, p):
#         #print( 'My_New_Mover.apply:', type(p) )
#         #print( 'This My_New_Mover apply...' )

#         cstopt = rosetta.protocols.enzdes.EnzdesBaseProtocol()
#         cstopt.set_scorefxn( self.sfxn )
#         cstopt.set_minimize_options(True, False, True, True) # check this fn signature for details

#         # this actuall runs the minimizer !
#         cstopt.cst_minimize(p, self.pt, True)

# cstopt = My_New_Mover(sfxn,pt)

# ## Note that a mover must have an apply function that takes a pose and does something to it
# ## in this case, cst opt derives from Enzdesbase protocol, but in the xml it's call from an option in
# ## the enzdesrepackmin mover... blah blah blah, long story short, this ends up working in pyrosetta since
# ## there are currently no setters for this option in the enzrepackmin mover... this needs to be fixed
# ## Now we string all of our movers together so that one mover can be fed into the generic monte carlo

# parsed = rosetta.protocols.moves.SequenceMover()
# #parsed.add_mover(pm) # state 2, nothing should have changed
# parsed.add_mover(predock)
# #parsed.add_mover(pm)  # state 3, only the ligand should have moved
# parsed.add_mover(cstopt)
# #parsed.add_mover(pm)  # state 4 , the constraints should be optimized/satisfied
# parsed.add_mover(enzdes)
# #parsed.add_mover(pm) # state 5, designing and packing within the shells
# parsed.add_mover(enzdes_wbb)
# #parsed.add_mover(pm) # state 6, now, the backbone might have minor perturbations (sometimes barely noticeable)
# ## These states track with running
# ## a simulation for testing using "mc.apply(p)"
# ## now we setup the Generic Monte Carlo object to run the simulation, this will run our sequence mover/parsedprotocol
# ## for X cycles, you can set the criterion (sfxn) and filters and other stuff

# # The XML line looks like this
# ##<GenericMonteCarlo name=multi_dd mover_name=dock_des filter_name=allcst trials=10 sample_type=low temperature=0.6 drift=1\/>

# mc = rosetta.protocols.simple_moves.GenericMonteCarloMover()
# mc.set_drift(True) # this sets drift for the maxtrials (not technically mc anymore)
# mc.set_maxtrials(1)  # CHANGE THIS TO 10 for real runs
# mc.set_sampletype('low')
# mc.set_temperature(0.6)
# mc.set_mover(parsed)
# mc.set_scorefxn(sfxn)
# mc.apply(p)

# # This is how you would normally run for nstruct 10
# # Note: This is not actually necessary depending on your goals, you can
# # also mc.apply(p), or store the energies over the simulation, or p.dump_pdb() etc.
# nstruct = 2
# job_output = 'test_output'
# jd = pyrosetta.PyJobDistributor(job_output, nstruct, sfxn)
# temp_pose = rosetta.core.pose.Pose()    # a temporary pose to export to PyMOL
# temp_pose.assign(p)
# counter = 0    # for pretty output to PyMOL

# while not jd.job_complete:

#     counter += 1

#     test_pose = rosetta.core.pose.Pose()
#     # set staring pose to input pose
#     test_pose.assign(p)
#     # change pose name for pretty viewing in PyMol
#     test_pose.pdb_info().name(job_output + '_' + str(counter))

#     # apply mc mover (with all of the stuff)
#     mc.apply(test_pose)

#     # have the jd output the resulting low energy model
#     jd.output_decoy(test_pose)

